# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f58889614f0>
├── 'a' --> tensor([[-0.9771, -0.5984,  1.6520],
│                   [-1.2725, -0.6649, -1.0213]])
└── 'x' --> <FastTreeValue 0x7f58889cdee0>
    └── 'c' --> tensor([[-1.7678,  0.1199, -0.2184, -0.3164],
                        [-1.4435,  2.1339, -0.3543,  0.9503],
                        [-0.2586, -0.9907,  1.0291, -1.4260]])

In [4]:
t.a

tensor([[-0.9771, -0.5984,  1.6520],
        [-1.2725, -0.6649, -1.0213]])

In [5]:
%timeit t.a

58.5 ns ± 0.043 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f58889614f0>
├── 'a' --> tensor([[ 0.6496,  0.4499, -0.6495],
│                   [-0.8372, -1.2271, -0.8792]])
└── 'x' --> <FastTreeValue 0x7f58889cdee0>
    └── 'c' --> tensor([[-1.7678,  0.1199, -0.2184, -0.3164],
                        [-1.4435,  2.1339, -0.3543,  0.9503],
                        [-0.2586, -0.9907,  1.0291, -1.4260]])

In [7]:
%timeit t.a = new_value

63.6 ns ± 0.0585 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.9771, -0.5984,  1.6520],
               [-1.2725, -0.6649, -1.0213]]),
    x: Batch(
           c: tensor([[-1.7678,  0.1199, -0.2184, -0.3164],
                      [-1.4435,  2.1339, -0.3543,  0.9503],
                      [-0.2586, -0.9907,  1.0291, -1.4260]]),
       ),
)

In [10]:
b.a

tensor([[-0.9771, -0.5984,  1.6520],
        [-1.2725, -0.6649, -1.0213]])

In [11]:
%timeit b.a

51.2 ns ± 0.0459 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.1220, -0.0200, -0.7205],
               [-0.4455,  0.8957,  0.4124]]),
    x: Batch(
           c: tensor([[-1.7678,  0.1199, -0.2184, -0.3164],
                      [-1.4435,  2.1339, -0.3543,  0.9503],
                      [-0.2586, -0.9907,  1.0291, -1.4260]]),
       ),
)

In [13]:
%timeit b.a = new_value

496 ns ± 0.189 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

832 ns ± 0.205 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 64.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

135 µs ± 345 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

137 µs ± 469 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f57dc66b4f0>
├── 'a' --> tensor([[[-0.9771, -0.5984,  1.6520],
│                    [-1.2725, -0.6649, -1.0213]],
│           
│                   [[-0.9771, -0.5984,  1.6520],
│                    [-1.2725, -0.6649, -1.0213]],
│           
│                   [[-0.9771, -0.5984,  1.6520],
│                    [-1.2725, -0.6649, -1.0213]],
│           
│                   [[-0.9771, -0.5984,  1.6520],
│                    [-1.2725, -0.6649, -1.0213]],
│           
│                   [[-0.9771, -0.5984,  1.6520],
│                    [-1.2725, -0.6649, -1.0213]],
│           
│                   [[-0.9771, -0.5984,  1.6520],
│                    [-1.2725, -0.6649, -1.0213]],
│           
│                   [[-0.9771, -0.5984,  1.6520],
│                    [-1.2725, -0.6649, -1.0213]],
│           
│                   [[-0.9771, -0.5984,  1.6520],
│                    [-1.2725, -0.6649, -1.0213]]])
└── 'x' --> <FastTreeValue 0x7f57dc66bfd0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32 µs ± 66.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f57dc61df40>
├── 'a' --> tensor([[-0.9771, -0.5984,  1.6520],
│                   [-1.2725, -0.6649, -1.0213],
│                   [-0.9771, -0.5984,  1.6520],
│                   [-1.2725, -0.6649, -1.0213],
│                   [-0.9771, -0.5984,  1.6520],
│                   [-1.2725, -0.6649, -1.0213],
│                   [-0.9771, -0.5984,  1.6520],
│                   [-1.2725, -0.6649, -1.0213],
│                   [-0.9771, -0.5984,  1.6520],
│                   [-1.2725, -0.6649, -1.0213],
│                   [-0.9771, -0.5984,  1.6520],
│                   [-1.2725, -0.6649, -1.0213],
│                   [-0.9771, -0.5984,  1.6520],
│                   [-1.2725, -0.6649, -1.0213],
│                   [-0.9771, -0.5984,  1.6520],
│                   [-1.2725, -0.6649, -1.0213]])
└── 'x' --> <FastTreeValue 0x7f57dc603970>
    └── 'c' --> tensor([[-1.7678,  0.1199, -0.2184, -0.3164],
                        [-1.4435,  2.1339, -0.3543,  0.9503],
                 

In [23]:
%timeit t_cat(trees)

29.5 µs ± 52.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

58.3 µs ± 112 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.9771, -0.5984,  1.6520],
                [-1.2725, -0.6649, -1.0213]],
       
               [[-0.9771, -0.5984,  1.6520],
                [-1.2725, -0.6649, -1.0213]],
       
               [[-0.9771, -0.5984,  1.6520],
                [-1.2725, -0.6649, -1.0213]],
       
               [[-0.9771, -0.5984,  1.6520],
                [-1.2725, -0.6649, -1.0213]],
       
               [[-0.9771, -0.5984,  1.6520],
                [-1.2725, -0.6649, -1.0213]],
       
               [[-0.9771, -0.5984,  1.6520],
                [-1.2725, -0.6649, -1.0213]],
       
               [[-0.9771, -0.5984,  1.6520],
                [-1.2725, -0.6649, -1.0213]],
       
               [[-0.9771, -0.5984,  1.6520],
                [-1.2725, -0.6649, -1.0213]]]),
    x: Batch(
           c: tensor([[[-1.7678,  0.1199, -0.2184, -0.3164],
                       [-1.4435,  2.1339, -0.3543,  0.9503],
                       [-0.2586, -0.9907,  1.0291, -1.4260]],
         

In [26]:
%timeit Batch.stack(batches)

77.8 µs ± 97.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.9771, -0.5984,  1.6520],
               [-1.2725, -0.6649, -1.0213],
               [-0.9771, -0.5984,  1.6520],
               [-1.2725, -0.6649, -1.0213],
               [-0.9771, -0.5984,  1.6520],
               [-1.2725, -0.6649, -1.0213],
               [-0.9771, -0.5984,  1.6520],
               [-1.2725, -0.6649, -1.0213],
               [-0.9771, -0.5984,  1.6520],
               [-1.2725, -0.6649, -1.0213],
               [-0.9771, -0.5984,  1.6520],
               [-1.2725, -0.6649, -1.0213],
               [-0.9771, -0.5984,  1.6520],
               [-1.2725, -0.6649, -1.0213],
               [-0.9771, -0.5984,  1.6520],
               [-1.2725, -0.6649, -1.0213]]),
    x: Batch(
           c: tensor([[-1.7678,  0.1199, -0.2184, -0.3164],
                      [-1.4435,  2.1339, -0.3543,  0.9503],
                      [-0.2586, -0.9907,  1.0291, -1.4260],
                      [-1.7678,  0.1199, -0.2184, -0.3164],
                      [-1.4435,  

In [28]:
%timeit Batch.cat(batches)

142 µs ± 477 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

310 µs ± 638 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
